<a href="https://colab.research.google.com/github/mikeroguez/experimento/blob/main/Tareas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1. Codigo para normalizar**

In [ ]:
import pandas as pd
import numpy as np
import html
from typing import Tuple, Optional

def get_semester_bounds(commitment_date: pd.Timestamp) -> Tuple[pd.Timestamp, pd.Timestamp]:
    """Determina los límites del semestre basado en la fecha de compromiso."""
    if pd.isnull(commitment_date):
        return pd.NaT, pd.NaT

    year, month = commitment_date.year, commitment_date.month

    if 2 <= month <= 7:
        return pd.Timestamp(year, 2, 1), pd.Timestamp(year, 7, 31)
    elif month == 1:
        return pd.Timestamp(year - 1, 8, 1), pd.Timestamp(year, 1, 31)
    else:
        return pd.Timestamp(year, 8, 1), pd.Timestamp(year + 1, 1, 31)

def calculate_delay_days(delivery_date: pd.Timestamp, commitment_date: pd.Timestamp) -> Optional[float]:
    """Calcula los días de retraso entre la fecha de entrega y la de compromiso."""
    if pd.isnull(delivery_date) or pd.isnull(commitment_date):
        return np.nan
    return (delivery_date - commitment_date).days

def process_data(input_path: str, output_path: str) -> pd.DataFrame:
    """Procesa el dataset de tareas académicas y guarda el resultado."""
    FINAL_COLUMNS = [
        'curso', 'groupKey', 'email', 'id_actividad', 'actividad',
        'fecha_compromiso', 'fecha_de_entrega', 'calificacion',
        'calificacion_normalizada', 'calificacion_estandarizada', 'es_atipico',
        'dias_anticipacion', 'fuera_de_rango', 'fue_entregada', 'fue_evaluada', 'tipo_entrega'
    ]

    # Cargar y preparar los datos
    df = pd.read_csv(input_path)
    df.rename(columns={
        'Fecha_compromiso': 'fecha_compromiso',
        'Fecha_de_entrega': 'fecha_de_entrega',
        'Calificacion': 'calificacion',
        'Actividad': 'actividad'
    }, inplace=True)

    # Parsear fechas sin tiempo (hora, minuto, segundo)
    df['fecha_compromiso'] = pd.to_datetime(df['fecha_compromiso'], errors='coerce').dt.date
    df['fecha_de_entrega'] = pd.to_datetime(df['fecha_de_entrega'], errors='coerce').dt.date

    # Convertir las fechas de nuevo a datetime para operaciones posteriores
    df['fecha_compromiso'] = pd.to_datetime(df['fecha_compromiso'])
    df['fecha_de_entrega'] = pd.to_datetime(df['fecha_de_entrega'])

    # Limpiar y normalizar la columna 'actividad'
    df['actividad'] = (
        df['actividad']
        .fillna("actividad desconocida")
        .apply(lambda x: html.unescape(str(x)))
        .str.replace(r'[^\x00-\x7F]+', '', regex=True)
    )

    # Identificar calificaciones atípicas
    df['es_atipico'] = ((df['calificacion'] > 100) & df['calificacion'].notna()).astype(int)

    # Manejar calificaciones faltantes
    df.loc[df['fecha_de_entrega'].isna() | (df['fecha_de_entrega'].notna() & df['calificacion'].isna()), 'calificacion'] = np.nan

    # Normalizar calificaciones
    max_scores = df[df['calificacion'] >= 0].groupby('id_actividad')['calificacion'].max()
    df['calificacion_normalizada'] = (
        df.apply(lambda x: (x['calificacion'] / max_scores.get(x['id_actividad'], np.nan)) * 10
                 if pd.notnull(x['calificacion']) and max_scores.get(x['id_actividad'], 0) > 0
                 else np.nan, axis=1)
        .round(2)
    )

    # Estandarizar calificaciones
    actividad_stats = df.groupby('id_actividad')['calificacion'].agg(mean='mean', std='std', count='count')
    df = df.merge(actividad_stats, left_on='id_actividad', right_index=True, how='left')
    df['calificacion_estandarizada'] = np.where(
        df['count'] < 2,
        np.nan,
        ((df['calificacion'] - df['mean']) / df['std'].replace(0, np.nan)).round(2)
    )

    # Calcular límites del semestre y días de anticipación
    df[['semester_start', 'semester_end']] = df['fecha_compromiso'].apply(lambda x: pd.Series(get_semester_bounds(x)))
    df['dias_anticipacion'] = (df['fecha_compromiso'] - df['fecha_de_entrega']).dt.days

    # Calcular si la entrega está fuera de rango y convertir a 0/1
    df['fuera_de_rango'] = df.apply(
        lambda x: (
            pd.notna(x['fecha_de_entrega']) and
            pd.notna(x['semester_start']) and
            pd.notna(x['semester_end']) and
            (x['fecha_de_entrega'] < x['semester_start'] or x['fecha_de_entrega'] > x['semester_end'])
        ),
        axis=1
    ).astype(int)  # Convertir a 0/1

    # Clasificar el tipo de entrega
    df['tipo_entrega'] = np.select(
        [
            df['fecha_de_entrega'].isna(),
            df['dias_anticipacion'] > 7,
            df['dias_anticipacion'].between(0, 7),
            df['dias_anticipacion'] < 0
        ],
        ['faltante', 'temprana', 'puntual', 'tardía'],
        default='desconocida'
    )

    # Indicadores de entrega y evaluación
    df['fue_entregada'] = df['fecha_de_entrega'].notna().astype(int)
    df['fue_evaluada'] = df['calificacion'].notna().astype(int)

    return df[FINAL_COLUMNS].copy()

if __name__ == "__main__":
    INPUT_PATH = "/content/drive/MyDrive/Doctorado/Experimentos/Datos/DSv4.0/1. Para tratar/tareas/tareas.csv"
    OUTPUT_PATH = "/content/drive/MyDrive/Doctorado/Experimentos/Datos/DSv4.0/1. Para tratar/tareas/tareas_normalizado.csv"

    processed_df = process_data(INPUT_PATH, OUTPUT_PATH)
    processed_df.to_csv(OUTPUT_PATH, index=False)
    print(f"Dataset procesado guardado en: {OUTPUT_PATH}")

Dataset procesado guardado en: /content/drive/MyDrive/Doctorado/Experimentos/Datos/DSv4.0/1. Para tratar/tareas/tareas_normalizado.csv


**2. Código para consolidar**

In [ ]:
import pandas as pd

# Cargar el dataset desde Google Drive
file_path = "/content/drive/MyDrive/Doctorado/Experimentos/Datos/DSv4.0/1. Para tratar/tareas/tareas_normalizado.csv"
data = pd.read_csv(file_path)

# Función para consolidar los datos con cálculos consistentes
def consolidate_data_fixed(data):
    # Agrupar y calcular métricas principales
    consolidated = data.groupby(['curso', 'email', 'groupKey']).agg(
        total_assignments=('id_actividad', 'count'),
        avg_assignment_normalized_score=('calificacion_normalizada', 'mean'),
        avg_assignment_estandard_score=('calificacion_estandarizada', 'mean'),
        max_assignment_normalized_score=('calificacion_normalizada', 'max'),
        min_assignment_normalized_score=('calificacion_normalizada', 'min'),
        max_assignment_estandard_score=('calificacion_estandarizada', 'max'),
        min_assignment_estandard_score=('calificacion_estandarizada', 'min'),
        score_variability=('calificacion_normalizada', 'std'),
        total_graded_assignments=('fue_evaluada', 'sum'),
        total_submitted_assignments=('fue_entregada', 'sum'),
        total_missing_assignments=('fue_entregada', lambda x: (x == 0).sum()),
        total_ungraded_assignments=('fue_evaluada', lambda x: (x == 0).sum()),
        total_out_of_range_assignments=('fuera_de_rango', 'sum'),
        total_late_assignments=('tipo_entrega', lambda x: (x == 'tardía').sum()),
        avg_submission_delay_days=(
            'dias_anticipacion',
            lambda x: x[data.loc[x.index, 'fue_entregada'] == 1].mean() if not x.empty else 0
        )
    ).reset_index()

    # Calcular tasas y métricas derivadas
    consolidated['assignment_submission_rate'] = (
        consolidated['total_submitted_assignments'] / consolidated['total_assignments']
    )
    consolidated['assignment_procrastination_rate'] = (
        consolidated['total_late_assignments'] / consolidated['total_submitted_assignments']
    )

    # Calcular rendimiento general ponderado
    w1, w2 = 0.5, 0.5
    consolidated['overall_assignment_performance'] = (
        (consolidated['avg_assignment_normalized_score'] * w1) +
        (consolidated['assignment_submission_rate'] * 10 * w2)
    )

    # Clasificar rendimiento general
    def classify_general_performance(overall_score):
        if overall_score >= 9.0:
            return 'Excelente'
        elif overall_score >= 8.0:
            return 'Bien'
        elif overall_score >= 6.0:
            return 'Suficiente'
        else:
            return 'Insuficiente'

    consolidated['overall_assignment_performance_classification'] = (
        consolidated['overall_assignment_performance'].apply(classify_general_performance)
    )

    # Revisar si todos los trabajos del grupo no fueron calificados
    group_status = consolidated.groupby(['curso', 'groupKey'])['total_graded_assignments'].sum().reset_index()
    group_status['delete'] = group_status['total_graded_assignments'].apply(lambda x: 1 if x == 0 else 0)

    # Unir el estado del grupo al DataFrame consolidado
    consolidated = pd.merge(
        consolidated, group_status[['curso', 'groupKey', 'delete']],
        on=['curso', 'groupKey'], how='left'
    )

    return consolidated

# Aplicar la función para consolidar los datos
corrected_consolidated_data = consolidate_data_fixed(data)

# Guardar los datos consolidados corregidos
output_path_corrected = "/content/drive/MyDrive/Doctorado/Experimentos/Datos/DSv4.0/1. Para tratar/tareas/tareas_consolidado.csv"
corrected_consolidated_data.to_csv(output_path_corrected, index=False)

#print(f"Datos consolidados guardados en: {output_path_corrected}")

# Mostrar los primeros registros
#print(corrected_consolidated_data.head())

print(f"Archivo consolidado guardado en: {output_path_corrected}")


**3. Código para limpiar**

In [ ]:
import pandas as pd

# Ruta del archivo de entrada y salida
input_file = "/content/drive/MyDrive/Doctorado/Experimentos/Datos/DSv4.0/1. Para tratar/tareas/tareas_consolidado.csv"
output_file = "/content/drive/MyDrive/Doctorado/Experimentos/Datos/DSv4.0/1. Para tratar/tareas/tareas_consolidado_cleaned.csv"

# Cargar el archivo consolidado
data = pd.read_csv(input_file)

# Filtrar registros donde delete = 1
def filter_data(data):
    return data[data['delete'] != 1].copy()

# Manejar valores nulos e imputar
def handle_missing_values(data):
    # Imputar valores nulos en las columnas relevantes
    fill_zero_columns = [
        'avg_assignment_normalized_score', 'avg_assignment_estandard_score',
        'max_assignment_normalized_score', 'min_assignment_normalized_score',
        'max_assignment_estandard_score', 'min_assignment_estandard_score',
        'score_variability', 'avg_submission_delay_days',
        'assignment_procrastination_rate', 'overall_assignment_performance'
    ]
    data[fill_zero_columns] = data[fill_zero_columns].fillna(0)

    # Crear una bandera para alumnos con una sola tarea entregada
    data['single_submission_flag'] = (data['total_submitted_assignments'] == 1).astype(int)

    return data

# Aplicar el pipeline
data_filtered = filter_data(data)
data_cleaned = handle_missing_values(data_filtered)

# Guardar el archivo limpio
data_cleaned.to_csv(output_file, index=False)

print(f"Archivo limpio guardado en: {output_file}")


Archivo limpio guardado en: /content/drive/MyDrive/Doctorado/Experimentos/Datos/DSv4.0/1. Para tratar/tareas/tareas_consolidado_cleaned.csv
